In [14]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

# data has one row per day for all counties
data = pd.read_csv("us_deaths.csv", parse_dates=True, dayfirst=False) 

us_states = data['Province/State'].unique()
print(us_states)
dpd=[]

for state in us_states:


print(agg_data)
print(agg_data.columns.values)
print(agg_data.keys())

us_states = agg_data['Province/State']
dates = agg_data['Date']

for state in us_states:
    dpd.append(agg_data[state])

fig, ax = plt.subplots()
im = ax.imshow(dpd)

# We want to show all ticks...
ax.set_xticks(np.arange(len(dates)))
ax.set_yticks(np.arange(len(us_states)))
# ... and label them with the respective list entries
ax.set_xticklabels(dates)
ax.set_yticklabels(us_states)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(us_states)):
    for j in range(len(dates)):
        text = ax.text(j, i, dpd[i, j],
                       ha="center", va="center", color="w")

ax.set_title("dpd")
fig.tight_layout()
plt.show()


SyntaxError: invalid syntax (<ipython-input-14-a7dc1d65d97b>, line 13)